In [65]:
import numpy as np
import pandas as pd
import extract
from tqdm import tqdm

In [66]:
d = extract.get_data(filename = "./data/EMMA_new_data_test.csv", usecols = 'alt')

In [67]:
d = extract.clean_data(d)

In [69]:
d

,cusip,tradedate,maturity,price,yield,amount,tradetype,coupon,name,state,issuetype,issuesource,rtg
0,13080SMW1,54.628472,3458.371528,98.250,4.095,85000,Purchase_from_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
1,13080SMW1,53.511806,3459.488194,100.000,3.999,15000,Sale_to_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
2,13080SMW1,53.490278,3459.509722,99.350,4.035,10000,Sale_to_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
3,13080SMW1,45.539583,3467.460417,100.000,3.999,20000,Sale_to_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
4,13080SMW1,45.531944,3467.468056,100.000,3.999,25000,Sale_to_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
5,13080SMW1,45.531944,3467.468056,97.940,4.112,100000,Purchase_from_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
6,13080SMW1,37.619444,3475.380556,100.000,3.999,10000,Sale_to_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
7,13080SMW1,37.615278,3475.384722,98.000,4.108,10000,Purchase_from_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
8,64577BTT1,53.666667,5224.333333,100.625,4.668,10000,Sale_to_Customer,4.75,new jersey st econ de,new jersey,ISSUE TYPE: revenue bonds,ISSUE SOURCE: APPROPRIATIONS,8
9,64577BTT1,40.694444,5237.305556,102.827,4.392,10000,Sale_to_Customer,4.75,new jersey st econ de,new jersey,ISSUE TYPE: revenue bonds,ISSUE SOURCE: APPROPRIATIONS,8


In [70]:
unique_cusip = list(d.cusip.unique())
dindex_set_1 = [] #d_index for first trade event
dindex_set_2 = [] #d_index for second trade event

dprice_data = []
holdtime_data = []
fixed_tradetype = 'purchase'
rtg_limit = 10
marketvalue_min = 10000
tradetype_dict = {'purchase': 'Sale_to_Customer', 'sell': 'Purchase_from_Customer'}

In [71]:
if 'amount' in d.columns:
    d_filter = d.loc[d['amount']*d['price']/100.0 >= float(marketvalue_min), :]
else:
    d_filter = d


In [72]:
d_filter

,cusip,tradedate,maturity,price,yield,amount,tradetype,coupon,name,state,issuetype,issuesource,rtg
0,13080SMW1,54.628472,3458.371528,98.250,4.095,85000,Purchase_from_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
1,13080SMW1,53.511806,3459.488194,100.000,3.999,15000,Sale_to_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
3,13080SMW1,45.539583,3467.460417,100.000,3.999,20000,Sale_to_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
4,13080SMW1,45.531944,3467.468056,100.000,3.999,25000,Sale_to_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
5,13080SMW1,45.531944,3467.468056,97.940,4.112,100000,Purchase_from_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
6,13080SMW1,37.619444,3475.380556,100.000,3.999,10000,Sale_to_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
8,64577BTT1,53.666667,5224.333333,100.625,4.668,10000,Sale_to_Customer,4.75,new jersey st econ de,new jersey,ISSUE TYPE: revenue bonds,ISSUE SOURCE: APPROPRIATIONS,8
9,64577BTT1,40.694444,5237.305556,102.827,4.392,10000,Sale_to_Customer,4.75,new jersey st econ de,new jersey,ISSUE TYPE: revenue bonds,ISSUE SOURCE: APPROPRIATIONS,8
10,64577BTT1,40.446528,5237.553472,100.999,4.621,10000,Sale_to_Customer,4.75,new jersey st econ de,new jersey,ISSUE TYPE: revenue bonds,ISSUE SOURCE: APPROPRIATIONS,8
11,64577BTT1,33.531944,5244.468056,97.500,4.995,50000,Purchase_from_Customer,4.75,new jersey st econ de,new jersey,ISSUE TYPE: revenue bonds,ISSUE SOURCE: APPROPRIATIONS,8


In [73]:
for s in tqdm(unique_cusip):
        #get all data points for given cusip
		I = (d_filter.cusip == s) & (d_filter.rtg <= rtg_limit)
		
		if 'tradeid' in d.columns:
			d_slice = d_filter.loc[I].sort_values(by = ['tradedate', 'tradeid'], ascending = [True, True])
		else:
			d_slice = d_filter.loc[I].sort_values(by = ['tradedate'], ascending = [True])

		if fixed_tradetype is not None:
			d_slice = d_slice.loc[d_slice.tradetype == tradetype_dict[fixed_tradetype], :]
			if d_slice.size == 0:
				continue
			else:
				for j in d_slice.index:
					d_entry = d_slice.loc[j,:]
					J = d_slice.tradedate > d_entry.tradedate
					dp = d_slice.loc[J].price.values
					if dp.shape[0] > 0:
						dp -= d_entry.price
						dindex_set_1 += dp.shape[0]*[j] #add row j of d, dp.shape[0] times
						dindex_set_2 += list(d_slice[J].index.values)
						holdtime = d_slice.loc[J].tradedate - d_entry.tradedate
						dprice_data += list(dp)
						holdtime_data += list(holdtime)

		else: #assume we want to collect buy->sell pairs
			#'Sale_to_Customer': 1, 'Purchase_from_Customer': 0
			d_purchase = d_slice[d_slice.tradetype == tradetype_dict['purchase']]
			d_sell = d_slice[d_slice.tradetype == tradetype_dict['sell']]
		
			if d_sell.size == 0 or d_purchase.size == 0:
				continue #no data points to compare for price changes
		
			else:
				#go through d_purchase sequentially and slice from d_sell for dtradedate values greater than current dtradedate
				for j in d_purchase.index:
					d_entry = d_purchase.loc[j,:]
					J = d_sell.tradedate > d_entry.tradedate
					dp = d_sell.loc[J].price.values
					if dp.shape[0] > 0:
						dp -= d_entry.price
						dindex_set_1 += dp.shape[0]*[j] #add row j of d, dp.shape[0] times
						dindex_set_2 += d_sell[J].index.values
						holdtime = d_sell.loc[J].tradedate - d_entry.tradedate
						dprice_data += list(dp)
						holdtime_data += list(holdtime)

100%|██████████| 3/3 [00:00<00:00, 40.17it/s]


In [74]:
print len(holdtime_data), len(dindex_set_1), len(dindex_set_2), len(dprice_data)

13 13 13 13


In [75]:
t = pd.DataFrame(np.vstack((dprice_data, holdtime_data)).T, columns = ['dprice', 'holdtime'])

In [76]:
t

,dprice,holdtime
0,0.000,7.912500
1,0.000,7.920139
2,0.000,15.892361
3,0.000,0.007639
4,0.000,7.979861
5,0.000,7.972222
6,0.905,7.111806
7,2.733,7.359722
8,0.531,20.331944
9,1.828,0.247917


In [77]:
t['d_index_1'] = dindex_set_1
t['d_index_2'] = dindex_set_2

In [78]:
t

,dprice,holdtime,d_index_1,d_index_2
0,0.000,7.912500,6,4
1,0.000,7.920139,6,3
2,0.000,15.892361,6,1
3,0.000,0.007639,4,3
4,0.000,7.979861,4,1
5,0.000,7.972222,3,1
6,0.905,7.111806,12,10
7,2.733,7.359722,12,9
8,0.531,20.331944,12,8
9,1.828,0.247917,10,9


In [62]:
#d_filter.pop('d_index_1')
#d_filter.loc[:,'d_index_1'] = d_filter.index
d_filter['d_index_1']

0                    0
1                    1
3                    3
4                    4
5                    5
6                    6
8                    8
9                    9
10                  10
11                  11
12                  12
13                  13
14                  14
d_index_1    d_index_1
Name: d_index_1, dtype: object

In [63]:
#joined_data = pd.concat([d_filter, t], axis=1, join='inner')
joined_data = t.merge(d_filter, how='left', on='d_index_1')

In [64]:
joined_data

,dprice,holdtime,d_index_1,d_index_2,cusip,tradedate,maturity,price,yield,amount,tradetype,coupon,name,state,issuetype,issuesource,rtg
0,0.000,7.912500,6,4,13080SMW1,37.619444,3475.380556,100.000,3.999,10000.0,Sale_to_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5.0
1,0.000,7.920139,6,3,13080SMW1,37.619444,3475.380556,100.000,3.999,10000.0,Sale_to_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5.0
2,0.000,15.892361,6,1,13080SMW1,37.619444,3475.380556,100.000,3.999,10000.0,Sale_to_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5.0
3,0.000,0.007639,4,3,13080SMW1,45.531944,3467.468056,100.000,3.999,25000.0,Sale_to_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5.0
4,0.000,7.979861,4,1,13080SMW1,45.531944,3467.468056,100.000,3.999,25000.0,Sale_to_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5.0
5,0.000,7.972222,3,1,13080SMW1,45.539583,3467.460417,100.000,3.999,20000.0,Sale_to_Customer,4.00,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5.0
6,0.905,7.111806,12,10,64577BTT1,33.334722,5244.665278,100.094,4.750,20000.0,Sale_to_Customer,4.75,new jersey st econ de,new jersey,ISSUE TYPE: revenue bonds,ISSUE SOURCE: APPROPRIATIONS,8.0
7,2.733,7.359722,12,9,64577BTT1,33.334722,5244.665278,100.094,4.750,20000.0,Sale_to_Customer,4.75,new jersey st econ de,new jersey,ISSUE TYPE: revenue bonds,ISSUE SOURCE: APPROPRIATIONS,8.0
8,0.531,20.331944,12,8,64577BTT1,33.334722,5244.665278,100.094,4.750,20000.0,Sale_to_Customer,4.75,new jersey st econ de,new jersey,ISSUE TYPE: revenue bonds,ISSUE SOURCE: APPROPRIATIONS,8.0
9,1.828,0.247917,10,9,64577BTT1,40.446528,5237.553472,100.999,4.621,10000.0,Sale_to_Customer,4.75,new jersey st econ de,new jersey,ISSUE TYPE: revenue bonds,ISSUE SOURCE: APPROPRIATIONS,8.0


In [20]:
tradetype_dict = {'purchase': 'Sale_to_Customer', 'sell': 'Purchase_from_Customer'}
fixed_tradetype = 'purchase'

In [21]:
d_slice_2 = d_slice.loc[d_slice.tradetype == tradetype_dict[fixed_tradetype], :]

In [22]:
d_slice_2

,cusip,tradedate,maturity,price,yield,amount,tradetype,coupon,name,state,issuetype,issuesource,rtg
6,13080SMW1,37.619444,3475.380556,100.0,3.999,10000,Sale_to_Customer,4.0,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
4,13080SMW1,45.531944,3467.468056,100.0,3.999,25000,Sale_to_Customer,4.0,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
3,13080SMW1,45.539583,3467.460417,100.0,3.999,20000,Sale_to_Customer,4.0,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
1,13080SMW1,53.511806,3459.488194,100.0,3.999,15000,Sale_to_Customer,4.0,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5


In [38]:
dindex_set_1 = [] #d_index for first trade event
dindex_set_2 = [] #d_index for second trade event
dprice_data = []
holdtime_data = []
for j in d_slice_2.index:
                    d_entry = d_slice_2.loc[j,:]
                    I = d_slice_2.tradedate > d_entry.tradedate
                    dp = d_slice_2.loc[I].price.values
                    if dp.shape[0] > 0:
                        dp -= d_entry.price
                        dindex_set_1 += dp.shape[0]*[j] #add row j of d, dp.shape[0] times
                        dindex_set_2 += list(d_slice_2[I].index.values)
                        holdtime = d_slice_2.loc[I].tradedate - d_entry.tradedate
                        dprice_data += list(dp)
                        holdtime_data += list(holdtime)


In [39]:
holdtime_data

[7.9124999999999943,
 7.9201388888888857,
 15.892361111111107,
 0.0076388888888914153,
 7.9798611111111128,
 7.9722222222222214]

In [40]:
dindex_set_1

[6, 6, 6, 4, 4, 3]

In [41]:
dindex_set_2

[4, 3, 1, 3, 1, 1]

In [37]:
d_slice_2

,cusip,tradedate,maturity,price,yield,amount,tradetype,coupon,name,state,issuetype,issuesource,rtg
6,13080SMW1,37.619444,3475.380556,100.0,3.999,10000,Sale_to_Customer,4.0,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
4,13080SMW1,45.531944,3467.468056,100.0,3.999,25000,Sale_to_Customer,4.0,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
3,13080SMW1,45.539583,3467.460417,100.0,3.999,20000,Sale_to_Customer,4.0,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5
1,13080SMW1,53.511806,3459.488194,100.0,3.999,15000,Sale_to_Customer,4.0,ca stwd cmntys-a-ref,california,ISSUE TYPE: revenue bonds,"ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.",5


In [42]:
d_slice_2.index

Int64Index([6, 4, 3, 1], dtype='int64')

In [43]:
d_slice.index

Int64Index([6, 4, 5, 3, 1, 0], dtype='int64')

In [45]:
dc.index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64')

In [46]:
d_filter.index

Int64Index([0, 1, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14], dtype='int64')

In [48]:
d_filter.values

array([['13080SMW1', 54.62847222222222, 3458.371527777778, 98.25, 4.095,
        85000, 'Purchase_from_Customer', 4.0, 'ca stwd cmntys-a-ref',
        'california', 'ISSUE TYPE: revenue bonds',
        'ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.', 5],
       ['13080SMW1', 53.511805555555554, 3459.4881944444446, 100.0,
        3.9989999999999997, 15000, 'Sale_to_Customer', 4.0,
        'ca stwd cmntys-a-ref', 'california', 'ISSUE TYPE: revenue bonds',
        'ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.', 5],
       ['13080SMW1', 45.53958333333333, 3467.460416666667, 100.0,
        3.9989999999999997, 20000, 'Sale_to_Customer', 4.0,
        'ca stwd cmntys-a-ref', 'california', 'ISSUE TYPE: revenue bonds',
        'ISSUE SOURCE: HLTH, HOSP, NURSHOME REV.', 5],
       ['13080SMW1', 45.53194444444444, 3467.4680555555556, 100.0,
        3.9989999999999997, 25000, 'Sale_to_Customer', 4.0,
        'ca stwd cmntys-a-ref', 'california', 'ISSUE TYPE: revenue bonds',
        'ISSUE SOURCE: HLTH, HOSP, 

In [53]:
s = extract.compile_price_change_data(dc, fixed_tradetype = 'purchase', marketvalue_min = 20000, rtg_limit = 10)

100%|██████████| 3/3 [00:00<00:00, 139.01it/s]


ValueError: Length of values does not match length of index